# **Generative Adverserial Networks**


Generative adversarial networks (GANs) are algorithmic architectures that use two neural networks, pitting one against the other (thus the “adversarial”) in order to generate new, synthetic instances of data that can pass for real data. They are used widely in image generation, video generation and voice generation.


![Generative-Adversarial-Networks-Framework.png](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.projectpro.io%2Farticle%2Fgenerative-adversarial-networks%2F811&psig=AOvVaw2CQiY5-PhadK26GYT1Up1o&ust=1703109274249000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCNjv08--nIMDFQAAAAAdAAAAABAI)



While most deep generative models are trained by maximizing log likelihood or a lower bound on log likelihood, GANs take a radically different approach that does not require inference or explicit calculation of the data likelihood. Instead, two models are used to solve a minimax game: a generator which samples data, and a discriminator which classifies the data as real or generated.In theory these models are capable of modeling an arbitrarily complex probability distribution.


## Objective:

The objective of creating a simple GAN using the MNIST fashion dataset with TensorFlow is to generate new images of clothing items that are similar to the ones in the training set. This can be done by training a generator model to create new images and a discriminator model to distinguish between real and fake images. The generator model is trained to maximize the likelihood of the discriminator model classifying its output as real, while the discriminator model is trained to minimize this likelihood. As the models are trained, they learn to generate more realistic images and to distinguish between real and fake images more accurately.

## **Training the network**

The important thing about training a GAN is that the two components should never be trained together. Rather the network is trained in two different phases,the first phase is for training the discriminator and updating the weights appropriately and in the next step the generator is trained while the discriminator training is disabled.

**Phase 1**
During phase one of training the generator is fed random data(in the form of a distribution) as noise. The generator creates some random images which are given to the discriminator. The discriminator also takes input from dataset of real images.
The discriminator learns to distinguish the real data from the fake ones by learning or assessing features from it's inputs. The discriminator outputs some probability and difference between the predicted results and the actual results are backpropagated through the network and the weights of the discriminator is updated.
Remember during this phase, the backpropagation stops at the end of the discriminator and the generator is not trained or updated.


**Phase 2**
In this phase, the generator produced batch of images are directly given as input to the discriminator. The real images are not given this time to the discriminator. The generator learns by tricking the discriminator into it outputting false positives. The discriminator outputs probabilities which are assessed against the actual results and the weights of of the generator are updated through backpropagation.
Remember here during backpropagation, the discriminator weights should not be updated and kept as they were before.


## Loss Function:

Absolutely! Let's dive into the math of the simple GAN loss functions:

**Discriminator Loss:**

The most common discriminator loss is **Binary Cross-Entropy (BCE)** with sigmoid activation. Denoting:

* x_i: A real data point
* G(z_i): A fake data point generated by the generator from noise z_i
* D(x_i): The discriminator's probability of x_i being real
* D(G(z_i)): The discriminator's probability of G(z_i) being real

The BCE loss for a single data point is:

- L_D(x_i) = -log(D(x_i)) - log(1 - D(G(z_i)))

This penalizes the discriminator when it mistakes a real point for fake or a fake point for real. To train the discriminator, we minimize the *average* loss over all data points:

- min_D Σ L_D(x_i)

**Generator Loss:**

The simplest generator loss is the **negative log of the discriminator's probability**:

- L_G(z_i) = -log(D(G(z_i)))

This directly encourages the generator to maximize the discriminator's belief that its generated data is real. Again, we minimize the average loss over all data points:

- min_G Σ L_G(z_i)

However, as mentioned before, this can lead to saturation issues.

**Alternative Loss Functions:**

* **Wasserstein GAN (WGAN) Loss:** This replaces BCE with a Wasserstein distance metric, leading to more stable training but removing the probability interpretation.
* **Non-Saturating GAN Loss:** This replaces the negative log with the log of the discriminator's probability, avoiding saturation but requiring careful hyperparameter tuning.

These are just a few examples, and the specific formulas can vary depending on the chosen activation functions and distance metrics. The key takeaway is that the discriminator and generator losses work together to train each other through a game of "cat and mouse," ultimately leading to realistic data generation by the generator.

Remember, this is just a starting point. Feel free to ask if you'd like me to delve deeper into any specific aspect of the math or explore other loss functions!


# Importing Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np

import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Conv2D,Dropout,Dense,Flatten,Conv2DTranspose,BatchNormalization,LeakyReLU,Reshape
import tensorflow as tf

# Laod Data

In [3]:
# Load Fashion MNIST dataset
fashion_mnist, info = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteI1MZX2/fashion_mnist-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteI1MZX2/fashion_mnist-test.tfrecord*...:   0%…

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
# Train/test split (optional)
train_data, test_data = fashion_mnist['train'], fashion_mnist['test']

# Generator:

In [15]:
def define_generator(latent_dim):
  # Input layer for noise vector
  z = keras.Input(shape=(latent_dim,))

  # Dense layers to generate image features
  h = keras.layers.Dense(7 * 7 * 256, use_bias=False)(z)
  h = keras.layers.BatchNormalization()(h)
  h = keras.layers.LeakyReLU()(h)
  h = keras.layers.Reshape((7, 7, 256))(h)
  h = keras.layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', use_bias=False)(h)
  h = keras.layers.BatchNormalization()(h)
  h = keras.layers.LeakyReLU()(h)
  h = keras.layers.Conv2DTranspose(28, kernel_size=3, strides=2, padding='same', use_bias=False, activation='tanh')(h)

  # Output layer for generated image

  image = h

  return keras.Model(z, image)


# Descriminator

In [17]:
def define_discriminator(image_shape):
  # Input layer for real or generated image
  image = keras.Input(shape=image_shape)

  # Conv2D layers to discriminate between real and fake
  h = keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', use_bias=False)(image)
  h = keras.layers.LeakyReLU()(h)
  h = keras.layers.Dropout(0.25)(h)
  h = keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', use_bias=False)(h)
  h = keras.layers.LeakyReLU()(h)
  h = keras.layers.Dropout(0.25)(h)
  h = keras.layers.Flatten()(h)

  # Output layer for real/fake prediction
  validity = keras.layers.Dense(1, activation='sigmoid')(h)

  return keras.Model(image, validity)

In [24]:
# Define latent dimension and image shape
latent_dim = 100
image_shape = (28, 28, 1)


In [25]:
# Define generator and discriminator models
generator = define_generator(latent_dim)
discriminator = define_discriminator(image_shape)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the MNIST fashion dataset
(x_train, _), (_, _) = keras.datasets.fashion_mnist.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255.0

# Create the generator model
generator = keras.Sequential([
    layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(100,)),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.2),
    layers.Reshape((7, 7, 256)),
    layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='tanh'),
])

# Create the discriminator model
discriminator = keras.Sequential([
    layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
    layers.LeakyReLU(alpha=0.2),
    layers.Dropout(0.3),
    layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(1),
    layers.Activation('sigmoid'),
])

# Compile the generator and discriminator models
generator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.compile(optimizer='adam', loss='binary_cross_entropy')

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
combined = keras.Sequential([generator, discriminator])

In [3]:
combined.compile(optimizer='adam', loss='binary_crossentropy')

In [5]:
x_train = x_train.reshape((x_train.shape[0], -1))